In [1]:
import pandas as pd
import http.client
from dotenv import load_dotenv
import json
import ast  # To safely evaluate the string as a dictionary
import matplotlib.pyplot as plt
import seaborn as sns

import os
import time

In [8]:
# connect weather API

load_dotenv(dotenv_path=r"C:\Users\taoti\GitHub\Hotel_Recommendations\.env")


print(os.environ["weather_api_host"])
print(os.environ["weather_api_key"])

weatherbit-v1-mashape.p.rapidapi.com
a29fa7550emshd22ad2dcdc1d6fap1bfd73jsnbc24356674f8


In [ ]:

# store data requested from API into dataframe for the convinience of analysis


# List of European locations (city: lat, lon)
locations = {
    "London, UK": (51.5074, -0.1278),
    "Paris, France": (48.8566, 2.3522),
    "Berlin, Germany": (52.5200, 13.4050),
    "Rome, Italy": (41.9028, 12.4964),
    "Madrid, Spain": (40.4168, -3.7038)
}

# Initialize connection
conn = http.client.HTTPSConnection(os.environ["weather_api_host"])
headers = {
    'x-rapidapi-key': os.environ["weather_api_key"],
    'x-rapidapi-host': os.environ["weather_api_host"]
}

# Create a list all_weather_data to store all records
weather_data_lst = []


def weather_extract(conn, locations):
    all_weather_data = []

    # Loop through locations
    for city, (lat, lon) in locations.items():
        try: 
            # define url as local variable
            url = f"/forecast/daily?lat={lat}&lon={lon}&units=metric&lang=en"

            # Request to the API, and get data as the response
            conn.request("GET", url, headers=headers)
            res = conn.getresponse()
            data = res.read()

            # Parse JSON and extract data
            weather_data = json.loads(data)
            print(f"Response for {city}: {weather_data}")
            
            if 'data' in weather_data:
                    records = weather_data['data']
                    # Add city information to each record
                    for record in records:
                        record['city'] = city
                    all_weather_data.extend(records)
            else:
                    print(f"No 'data' key found for {city}. Skipping...")
                    time.sleep(1)  # Wait 1 second between requests
        except Exception as e:
            print(f"Error fetching data for {city}: {e}")
            time.sleep(5)  # Wait 5 seconds before retrying
    return all_weather_data

# call function weather_extract()
weather_data_lst = weather_extract(conn, locations)

# Create DataFrame from lsit all_weather_data
df = pd.DataFrame(weather_data_lst)

# Display the DataFrame
print(df.head())

# Save the DataFrame to a CSV file
df.to_csv("weather_data.csv", index=False)